Pre-trained models like BERT are initially trained on a large corpus of text (e.g., Wikipedia, BookCorpus) to learn general language representations. **Transfer learning** involves fine-tuning these pre-trained models on specific task, leveraging pre-exisitng knowledge to achieve better performance with less data and training time. 


## Understanding BERT's Architecture
- Bidirectional: considers context from both left and right of a token
- Transformer Encoder: Utilizes self-attention mechanism to build representations of words based on their context.

## Task-Specific Fine-Tuning For Similarity Detection
Each sample in the dataset consists of a pair of business names and a label indicating their similarity (0 for dissimilar,1 for similar). 
**Tokenization:** Tokenize the pairs of names using BERT's tokenizer. This includes adding special token ([CLS], [SEP]) to distingguish separate names

Example:

In [ ]:
inputs = tokenizer(name1, name2, return_tensor = 'pt', padding = 'max_length', truncation = True,  max_length = 128)

- We use the pre-trained BERT model to encode the input pairs. 
- Aggregate the token embeddings (e.g., mean pooling) to obtain a fixed-size representation. 
- Add a linear layer to map the pooled representation to similarity score.

Example:

In [ ]:
class Bus

**Loss function and Optimization**: Use a suitable loss function for binary classification (e.g., Binary Cross-Entropy Loss).

**Training Loop**:
- Forward pass: compute the similarity score for input pairs
- Loss computation: Calculate the loss between predicted similarity scores and actual labels. 
- Backward Pass and Optimization: Update model parameters based on the gradient.

### Why Tokenize Pairs Together?

**Contextual Relationship**:
By tokenizing pairs together and feeding them into the model, BERT can consider the interation between the two names in a single forward pass. This allows the model to learn more about their relationship, which is crucial for taks like similarity detection. 
When embeddings are generated separately, the model cannot leverage the full context of both names together. Comparing separate embeddings via cosine similarity captures some relational informatio but lacks the deeper interaction modeling that joint tokenization provides. 